# 체크용

In [2]:
import pandas as pd
import os

df_match = pd.read_csv('ftn+_age_match_20201224-Copy1.csv',sep=',')
print(len(df_match))

fs=os.listdir('../../../cranberry2/Preprocessing/vital_data/PPG_100Hz_ECG_100Hz_pacu_5min/')


df_match['NRS_time']=pd.to_datetime(df_match['NRS_time'])
df_match = df_match.rename(columns={'Unnamed: 0':'index'})
#df_match.reset_index(inplace=True, drop=True)

print(len(df_match))
'''vital -> csv
df_match['path']=[f_name.split('.')[0]+".csv" for f_name in df_match['path'].values.flatten()] '''

nonlist=[]
yeslist=[]
srate= 100
for i in range(len(df_match)):
    filename = str(df_match.loc[i,'Value'])+','+str(df_match.loc[i,'index'])+','+ df_match.loc[i,'path']
    if not os.path.exists('../../../cranberry2/Preprocessing/vital_data/PPG_100Hz_ECG_100Hz_pacu_5min/'+filename):
        nonlist.append(df_match.loc[i,'path'])
    else:
        yeslist.append(df_match.loc[i,'path'])
        
len(fs), len(nonlist),len(yeslist)

10156


8804

In [16]:
import pickle

cid = pickle.load(open('../../DL_model/caseids_diff','rb'))
cid = [e[:-3]+'vital' for e in cid]

existed = []
still=[]
for p in nonlist:
    if p in cid:
        existed.append(p)
    if p in yeslist:
        still.append(p)
len(cid),len(existed),len(still)

(591, 190, 1132)

# ECG 250Hz

In [ ]:
import pandas as pd
import datetime
import time
import pickle
import os
import numpy as np
import vitaldb

df_match = pd.read_csv('ftn+_age_match_20201224-Copy1.csv',sep=',')
df_match['NRS_time']=pd.to_datetime(df_match['NRS_time'])
df_match = df_match.rename(columns={'Unnamed: 0':'index'})
#df_match.reset_index(inplace=True, drop=True)

print(len(df_match))
'''vital -> csv
df_match['path']=[f_name.split('.')[0]+".csv" for f_name in df_match['path'].values.flatten()] '''

srate= 250
for i in range(len(df_match)-1,-1,-1):
    filename = str(df_match.loc[i,'Value'])+','+str(df_match.loc[i,'index'])+','+ df_match.loc[i,'path']
    if not os.path.exists('../../../cranberry2/Preprocessing/vital_data/ECG_250Hz_pacu_5min/'+filename):
        print(i, end=' ')
        #try: 
        rec_path = '../../../cranberry2/Preprocessing/vital_data/pacu_ecg_pleth/pacu_ecg_pleth/'+df_match.loc[i,'path']
        #print(rec_path)
        #print('1')
        vfile_ecg = vitaldb.vital_recs_time(rec_path,['SNUADCW/ECG_II'],interval=0.004).flatten()

        #print(vfile)
        if len(vfile_ecg[0])==0 :
            print('index ',df_match.loc[i,'index'],' OPID ',df_match.loc[i,'opid'],'empty vital file')
        else:
            Abs_time = [vfile_ecg[1]+datetime.timedelta(hours=9)+datetime.timedelta(seconds=i/srate) for i in range(len(vfile_ecg[0]))]
            #print(vfile[1])
            dics = {'Abs_time':Abs_time,'ECG':vfile_ecg[0]}
            df_vital = pd.DataFrame(data=dics)
            end_idx_list = df_vital[(df_vital['Abs_time'] > df_match.loc[i,'NRS_time'] - datetime.timedelta(seconds=1)) & (df_vital['Abs_time'] < df_match.loc[i,'NRS_time'] + datetime.timedelta(seconds=1))].index.tolist()

            if len(end_idx_list)==0:
                print('index ',i,' OPID ',df_match.loc[i,'opid']," no vital data at NRS time")

            else:
                end_idx=end_idx_list[0]+1
                start_idx= end_idx - 5 * 60 * srate
                start_idx_n = max(0,start_idx)

                ext = df_vital[start_idx_n:end_idx]
                extr=ext.dropna(subset=['ECG'])
                extr=extr.drop_duplicates(subset=['ECG'])

                if len(extr)>1:
                    if start_idx <0 :
                        fit = (-1)*end_idx 
                        extraction = pd.DataFrame(index=range(5*60*srate),columns=['Abs_time','ECG'])
                        extraction[fit:]=ext

                    else:
                        extraction = ext
                    with open('../../../cranberry2/Preprocessing/vital_data/ECG_250Hz_pacu_5min/'+filename,'wb') as f:
                        pickle.dump(extraction, f)
                    print('...done')
                else:
                    print('...결측치/직선 데이터')
        #except:
            #print('error: ',i,df_match.loc[i,'opid'],df_match.loc[i,'path'])

10156
10132 ...done
9834 ...결측치/직선 데이터
9767 ...done
9720 ...done
9614 ...done
9607 ...done
9433 ...done
9104 ...done
9054 ...done
8789 ...결측치/직선 데이터
8788 ...done
8745 ...done
8609 ...done
8605 ...결측치/직선 데이터
8591 ...done
8499 ...결측치/직선 데이터
8498 ...결측치/직선 데이터
8496 ...결측치/직선 데이터
8490 ...결측치/직선 데이터
8396 ...done
8391 ...done
8390 ...done
8356 ...done
8323 ...done
8316 ...done
8291 ...done
8287 ...done
8284 ...done
8278 ...done
8260 ...done
8253 ...done
8252 ...done
8248 ...done
8247 ...done
8239 ...done
8233 ...done
8223 ...done
8221 ...done
8214 ...done
8212 ...done
8204 ...done
8201 ...done
8198 ...done
8196 ...done
8190 ...done
8174 ...done
8166 ...done
8161 ...done
8159 ...done
8151 ...done
8146 ...done
8145 ...done
8143 ...결측치/직선 데이터
8131 ...done
8116 ...done
8112 ...done
8108 ...done
8107 ...done
8106 ...done
8098 ...done
8097 ...done
8096 ...done
8092 ...done
8071 ...done
8066 ...done
8064 ...done
8056 ...done
8053 ...done
8048 ...done
8044 ...done
8033 ...done
8029 ...결측치/직선 데이터
802

4180 ...done
4164 ...done
4149 ...done
4147 ...done
4143 ...done
4132 ...done
4122 ...done
4112 ...done
4111 ...done
4110 ...done
4102 ...done
4100 ...done
4099 ...done
4095 ...done
4081 ...done
4072 ...done
4071 ...done
4057 ...done
4048 ...done
4043 ...done
4041 ...done
4036 ...done
4032 ...결측치/직선 데이터
4030 ...done
4019 ...done
4017 ...done
4012 ...done
4000 ...done
3989 ...done
3985 ...done
3984 ...done
3981 ...done
3978 ...done
3972 ...done
3970 ...done
3966 ...done
3950 ...done
3949 ...done
3947 ...done
3943 ...done
3935 ...done
3924 ...done
3914 ...done
3907 ...결측치/직선 데이터
3902 ...done
3896 ...done
3895 ...done
3890 ...done
3886 ...done
3881 ...done
3880 ...done
3876 ...done
3871 ...done
3858 ...done
3855 ...done
3850 ...done
3845 ...done
3838 ...done
3834 ...done
3833 ...done
3831 ...done
3827 ...done
3822 ...done
3817 ...done
3797 ...done
3793 ...done
3792 ...done
3790 ...done
3777 ...done
3772 ...결측치/직선 데이터
3761 ...done
3759 ...done
3741 ...done
3740 ...done
3736 ...done
3720 ..

In [2]:
import pandas as pd
import datetime
import time
import pickle
import os
import numpy as np
import vitaldb

df_match = pd.read_csv('ftn+_age_match_20201224-Copy1.csv',sep=',')
df_match['NRS_time']=pd.to_datetime(df_match['NRS_time'])
df_match = df_match.rename(columns={'Unnamed: 0':'index'})
#df_match.reset_index(inplace=True, drop=True)

print(len(df_match))
'''vital -> csv
df_match['path']=[f_name.split('.')[0]+".csv" for f_name in df_match['path'].values.flatten()] '''


error_list=[]
srate= 250
for i in range(2000,4000):
    filename = str(df_match.loc[i,'Value'])+','+str(df_match.loc[i,'index'])+','+ df_match.loc[i,'path']
    vital_path = '../../../cranberry2/Preprocessing/vital_data/ECG_250Hz_pacu_5min/'+filename
    
    if os.path.exists(vital_path):
    #if not os.path.exists('../../../cranberry2/Preprocessing/vital_data/ECG_250Hz_pacu_5min/'+filename):
        print(i, end=' ')

        rec_path = '../../../cranberry2/Preprocessing/vital_data/pacu_full_vital/'+df_match.loc[i,'path']
        
        
        # vitaldb 에러에 해당되는 케이스
        df_vital = pickle.load(open(vital_path,'rb'))
        
        if np.sum(~np.isnan(df_vital['ECG'].tolist())) != 0:
            if np.nanmean(df_vital['ECG'].tolist()) > -4:
                print('already done')
                continue
        
        
        error_list.append(filename)
        print(df_match.loc[i,'path'], end='... ')
        
        vfile_ecg = vitaldb.vital_recs_time(rec_path,['SNUADCW/ECG_II'],interval=0.004).flatten()

        
        if len(vfile_ecg[0])==0 :
            print('index ',df_match.loc[i,'index'],' OPID ',df_match.loc[i,'opid'],'empty vital file')
            
        else:
            Abs_time = [vfile_ecg[1]+datetime.timedelta(hours=9)+datetime.timedelta(seconds=i/srate) for i in range(len(vfile_ecg[0]))]
            #print(vfile[1])
            dics = {'Abs_time':Abs_time,'ECG':vfile_ecg[0]}
            df_vital = pd.DataFrame(data=dics)
            end_idx_list = df_vital[(df_vital['Abs_time'] > df_match.loc[i,'NRS_time'] - datetime.timedelta(seconds=1)) & (df_vital['Abs_time'] < df_match.loc[i,'NRS_time'] + datetime.timedelta(seconds=1))].index.tolist()

            
            if len(end_idx_list)==0:
                print('index ',i,' OPID ',df_match.loc[i,'opid']," no vital data at NRS time")

            else:
                end_idx=end_idx_list[0]+1
                start_idx= end_idx - 5 * 60 * srate
                start_idx_n = max(0,start_idx)

                ext = df_vital[start_idx_n:end_idx]
                extr=ext.dropna(subset=['ECG'])
                extr=extr.drop_duplicates(subset=['ECG'])

                if len(extr)>1:
                    if start_idx <0 :
                        fit = (-1)*end_idx 
                        extraction = pd.DataFrame(index=range(5*60*srate),columns=['ECG'])
                        

                        col_ecg = np.full(5*60*srate, np.nan)
                        col_ecg[fit:] = ext['ECG'][fit:].tolist()                    
                        extraction['ECG'] = col_ecg                        
                        

                    else:
                        extraction = ext
                        
                    with open('../../../cranberry2/Preprocessing/vital_data/ECG_250Hz_pacu_5min/'+filename,'wb') as f:
                        pickle.dump(extraction, f)
                    print('...done')
                else:
                    print('...결측치/직선 데이터')
                
        #except:
            #print('error: ',i,df_match.loc[i,'opid'],df_match.loc[i,'path'])

10156
2000 already done
2001 already done
2002 already done
2003 PACU1_8_190830_165827.vital... ...done
2004 already done
2005 already done
2006 already done
2007 already done
2008 already done
2009 already done
2010 already done
2011 already done
2012 already done
2013 already done
2014 PACU1_1_190902_131344.vital... ...done
2015 already done
2016 already done
2017 already done
2018 already done
2019 already done
2020 already done
2021 already done
2023 already done
2024 already done
2025 already done
2026 already done
2027 already done
2028 already done
2029 already done
2030 already done
2031 already done
2032 PACU1_4_190902_234827.vital... ...done
2033 already done
2034 already done
2035 PACU1_5_190903_090959.vital... ...done
2036 already done
2037 already done
2038 already done
2039 already done
2040 already done
2041 already done
2042 already done
2043 already done
2044 already done
2045 already done
2046 already done
2047 already done
2048 already done
2049 already done
2050 alr

2415 already done
2416 already done
2417 already done
2418 already done
2419 already done
2420 already done
2421 already done
2422 already done
2423 already done
2424 already done
2425 already done
2427 already done
2428 already done
2429 already done
2430 already done
2431 already done
2432 already done
2433 already done
2434 already done
2435 already done
2436 already done
2437 already done
2438 already done
2439 already done
2440 already done
2441 already done
2442 already done
2443 already done
2444 already done
2445 PACU1_6_191018_150306.vital... ...done
2446 already done
2447 PACU1_5_191018_153230.vital... ...done
2448 already done
2449 already done
2450 already done
2451 already done
2452 already done
2453 PACU1_6_191018_155751.vital... ...done
2454 already done
2455 PACU1_7_191018_165330.vital... ...done
2456 already done
2457 already done
2458 already done
2459 already done
2460 already done
2461 already done
2462 already done
2463 already done
2464 already done
2465 already d

2827 already done
2828 already done
2829 already done
2830 already done
2831 already done
2832 already done
2833 already done
2834 already done
2835 already done
2836 already done
2837 already done
2838 already done
2839 already done
2840 already done
2841 PACU1_7_200109_094409.vital... ...done
2842 already done
2843 already done
2844 already done
2845 already done
2846 already done
2847 already done
2848 already done
2849 already done
2850 already done
2851 already done
2852 already done
2853 already done
2854 PACU1_5_200109_115336.vital... ...done
2855 already done
2856 already done
2857 already done
2858 PACU1_7_200109_125250.vital... ...done
2859 already done
2860 already done
2861 already done
2862 PACU1_12_200109_150545.vital... ...done
2863 already done
2864 already done
2865 already done
2866 already done
2867 already done
2868 already done
2869 already done
2870 already done
2871 already done
2872 already done
2873 already done
2874 already done
2875 already done
2876 already 

3245 already done
3246 already done
3247 already done
3248 already done
3249 already done
3250 already done
3251 already done
3252 already done
3253 already done
3254 already done
3255 already done
3256 already done
3257 already done
3258 already done
3259 already done
3260 already done
3261 already done
3262 already done
3263 already done
3264 already done
3265 already done
3266 PACU1_2_200127_094846.vital... ...done
3267 already done
3268 already done
3269 already done
3270 already done
3271 already done
3272 already done
3273 PACU1_1_200128_132348.vital... ...done
3274 already done
3275 already done
3276 already done
3277 already done
3278 already done
3279 already done
3280 already done
3281 already done
3282 already done
3283 already done
3284 PACU1_8_200129_095251.vital... ...done
3285 already done
3286 already done
3287 already done
3288 already done
3289 already done
3290 already done
3291 already done
3292 already done
3293 already done
3294 already done
3295 already done
3296

3651 PACU1_3_200213_121335.vital... ...done
3652 already done
3653 PACU1_5_200213_123443.vital... ...done
3654 already done
3655 already done
3656 already done
3657 already done
3658 PACU1_7_200213_140306.vital... ...done
3659 PACU1_1_200213_140923.vital... ...done
3660 already done
3661 PACU1_6_200213_142252.vital... ...done
3662 already done
3663 already done
3664 already done
3665 already done
3666 already done
3667 already done
3668 already done
3669 already done
3670 already done
3671 already done
3672 already done
3673 already done
3674 already done
3675 already done
3676 already done
3677 already done
3678 already done
3679 already done
3680 already done
3681 already done
3682 already done
3683 already done
3684 already done
3685 already done
3686 already done
3687 already done
3688 already done
3689 already done
3690 already done
3691 already done
3692 already done
3693 PACU1_5_200214_094414.vital... ...done
3694 already done
3695 already done
3696 already done
3697 already don

# NIBP PACU pickle로 저장하기

In [4]:
import pandas as pd
import datetime
import time
import pickle
import os
import numpy as np
from pyvital2 import vitaldb

df_match = pd.read_csv('match_drug_arr_exclusion_20201110.csv',sep=',')
df_match['NRS_time']=pd.to_datetime(df_match['NRS_time'])
df_IDpath = pd.read_csv('opid_fileid.csv')
#df_match['path']=[f_name.split('.')[0]+".csv" for f_name in df_match['path'].values.flatten()]

for j in range(len(df_IDpath)):
    #try: 
    #opid match
    idx_list = df_match[df_match['opid']==df_IDpath.loc[j,'opid']].index.tolist()
    i = min(idx_list)

    rec_path = 'preop_vital/preop_vital/' + df_IDpath.loc[j,'fileid']
    df_IDpath.loc[j, 'file_path'] = df_match.loc[i, 'path']
    df_IDpath.loc[j, 'NRS'] = df_match.loc[i, 'Value']
    df_IDpath.loc[j, 'NRS_time'] = df_match.loc[i, 'NRS_time']
print(df_IDpath)
print(df_match)

           opid                  fileid                    file_path   NRS  \
0     190408044  A2_190408_103443.vital  PACU1_1_190408_115004.vital   6.0   
1     190408106  E3_190408_180036.vital  PACU1_1_190408_202459.vital   7.0   
2     190409012  A2_190409_073654.vital  PACU1_2_190409_110712.vital   4.0   
3     190409044  D5_190409_101311.vital  PACU1_3_190409_114333.vital   6.0   
4     190409057  F4_190409_105106.vital  PACU1_1_190409_133948.vital   8.0   
...         ...                     ...                          ...   ...   
4109  201030106  R9_201030_145209.vital  PACU1_1_201030_161119.vital   6.0   
4110  201030057  E6_201030_131959.vital  PACU1_3_201030_195639.vital   7.0   
4111  201030082  R2_201030_185910.vital  PACU1_5_201030_203752.vital   6.0   
4112  201030023  C4_201030_185047.vital  PACU1_1_201030_211816.vital   6.0   
4113  201031004  R9_201031_084856.vital  PACU1_3_201031_131135.vital  10.0   

                NRS_time  
0    2019-04-08 12:00:00  
1    2019

In [ ]:
import pandas as pd
import datetime
import time
import pickle
import os
import numpy as np
from pyvital2 import vitaldb

'''vital -> csv
df_match['path']=[f_name.split('.')[0]+".csv" for f_name in df_match['path'].values.flatten()] '''

srate= 250
for i in range(len(df_match)):
    bedno = df_match.loc[i,'path'].split('_')[1]
    if int(bedno) <4:
        filename = str(df_match.loc[i,'Value'])+','+str(i)+','+ df_match.loc[i,'path']
        if not os.path.exists('vital_data/NIBP_250Hz_pacu_2min_pickle/'+filename):
            rec_path = 'PACU_vital/PACU1_1-3/'+df_match.loc[i,'path']
            vfile = vitaldb.vital_recs_time(rec_path,['Solar8000/NIBP_MBP'],interval=0.004).flatten()
            #print(vfile)
            if len(vfile[0])==0:
                print('index ',i,' OPID ',df_match.loc[i,'opid'],'empty NIBP')
            else:
                Abs_time = [vfile[1]+datetime.timedelta(hours=9)+datetime.timedelta(seconds=i/srate) for i in range(len(vfile[0]))]
                #print(vfile[1])
                dics = {'Abs_time':Abs_time,'NIBP':vfile[0]}
                df_NIBP = pd.DataFrame(data=dics)
                end_idx_list = df_NIBP[(df_NIBP['Abs_time'] > df_match.loc[i,'NRS_time'] - datetime.timedelta(seconds=1)) & (df_NIBP['Abs_time'] < df_match.loc[i,'NRS_time'] + datetime.timedelta(seconds=1))].index.tolist()

                if len(end_idx_list)==0:
                    print('index ',i,' OPID ',df_match.loc[i,'opid']," no vital data at NRS time")                  
                else:
                    end_idx=end_idx_list[0]
                    start_idx= max(0,end_idx - 2 * 60 * srate)
                    extraction = df_NIBP.loc[start_idx:end_idx]
                    with open('vital_data/NIBP_250Hz_pacu_2min_pickle/'+filename,'wb') as f:
                        pickle.dump(extraction, f)
                        
#이건 2분보다 짧아도 저장함

# NIBP PACU pickle로 저장하기

In [ ]:
import pandas as pd
import pickle
import datetime
import time
from pyvital2 import vitaldb
import numpy as np
import os

'''vital -> csv
df_match['path']=[f_name.split('.')[0]+".csv" for f_name in df_match['path'].values.flatten()] '''

srate= 250
for i in range(len(df_IDpath)):
    filename = df_IDpath.loc[i,'file_path']
    if not os.path.exists('vital_data/NIBP_250Hz_preop_2min_pickle/'+filename):
        rec_path = 'preop_vital/preop_vital/'+ df_IDpath.loc[i,'fileid']
        vfile = vitaldb.vital_recs(rec_path,['Solar8000/NIBP_MBP'],interval=0.004).flatten()[0:2*60*250]

        #print(vfile)
        if len(vfile) ==0 :
            print('index ',i,' OPID ',df_IDpath.loc[i,'opid'],'empty NIBP')
        else:
            dics = {'NIBP':vfile}
            df_NIBP = pd.DataFrame(data=dics)

            extraction = df_NIBP
            with open('vital_data/NIBP_250Hz_preop_2min_pickle/'+filename,'wb') as f:
                pickle.dump(extraction, f)
            print(i,'/',len(df_IDpath),' save')